# consistency of self calls
We are trying to see how reliably self echoes are heard and how consistent they are in the information they provide as oppposed to information coming from other direct calls/ echeos from their direct calls

Let us first load the data. 

In [17]:
import sys
import os 
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np

os.chdir("/home/adityamoger/Documents/GitHub/dynamic_model_of_cocktail_party_nightmare/")
print(os.getcwd())
sys.path.append("./exploratory_analysis/")
sys.path.append("./dynamic_model/")
from snr_implementation import given_sound_objects_return_detected_sounds


from read_simulation_output import read_data_per_simulation_per_bat
from snr_implementation import given_sound_objects_return_detected_sounds
from supporting_files.utilities import make_dir

plt.style.use("dark_background")

/home/adityamoger/Documents/GitHub/dynamic_model_of_cocktail_party_nightmare


In [ ]:
FOCAL_BATS = [1, 20]
NUM_BATS = 50
TEMPORAL_MASKING_DIR = "./exploratory_analysis/temporal_masking_fn.csv"
DIRECTORY = f"/media/adityamoger/T7 Shield1/consistency_of_calls/DATA_effect_of_group_size/{NUM_BATS}/iteration_number_3/"
DIRECTORY_STORE_PLOTS = "./exploratory_analysis/plots/"
INCLUDE_DIRECT_SOUNDS_ARR = [True]
AZIMUTH_THRESHOLDS = [np.pi / 6, np.pi / 2, np.pi]

In [18]:
for INCLUDE_DIRECT_SOUNDS in INCLUDE_DIRECT_SOUNDS_ARR:
    for FOCAL_BAT in FOCAL_BATS:
        for AZIMUTH in AZIMUTH_THRESHOLDS:

            track_unique_reflections = []
            sounds_to_plot = []
            times_of_sounds_to_plot = []
            delta_t = []
            OUTPUT_DIR = DIRECTORY + f"/{FOCAL_BAT}/"
            received_sounds_sorted_by_time = read_data_per_simulation_per_bat(
                OUTPUT_DIR, "received"
            )

            cm = plt.get_cmap("gist_rainbow")


            nan_matrix = (
                np.zeros(
                    shape=(len(received_sounds_sorted_by_time), NUM_COLORS - 1 + 4)
                )
                * np.nan
            )

            for i, frame in enumerate(received_sounds_sorted_by_time):
                heard_sounds = given_sound_objects_return_detected_sounds(
                    sound_objects=frame,
                    time_threshold_post_call=0.035,
                    angle_threshold=AZIMUTH,
                    dir_of_temporal_masking_fn_file=TEMPORAL_MASKING_DIR,
                    minimum_sound_detection_fraction=0.75,
                    focal_bat=FOCAL_BAT,
                    include_direct_sounds=INCLUDE_DIRECT_SOUNDS,
                )

                for sound_object in heard_sounds:

                    time = (
                        np.array(sound_object["occurance_times"])[0]
                        - sound_object["bat_last_call_time"]
                        + i * 0.04
                    )
                    theta = sound_object["bat_direction"].angle_between(
                        sound_object["incident_direction"]
                    )

                    if sound_object["type"] == "echo":
                        marker = "x"
                    else:
                        marker = "o"

                    if sound_object["emitter_id"] == FOCAL_BAT:
                        reflected_from = sound_object["reflected_from"]

                        if reflected_from not in track_unique_reflections:
                            track_unique_reflections.append(reflected_from)
                            times_of_sounds_to_plot.append([])
                            sounds_to_plot.append([])
                            delta_t.append([])
                        index = track_unique_reflections.index(reflected_from)
                        sounds_to_plot[index].append(theta)
                        times_of_sounds_to_plot[index].append(time)
                        delta_t[index].append(
                            np.array(sound_object["occurance_times"])[0]
                            - sound_object["bat_last_call_time"]
                        )
                        nan_matrix[i, index] = theta

            nan_matrix = nan_matrix[:, ~np.isnan(nan_matrix).all(0)]
            nan_matrix_rows = track_unique_reflections.copy()
            extender = [np.nan] * (NUM_BATS - 1 + 4 - len(nan_matrix_rows))
            nan_matrix_rows.extend(extender)
            # fig, axs = plt.subplots(
            #     1,
            #     1,
            #     figsize=(34, 14),
            # )
            plt.figure(figsize=(40, 10))
            plt.imshow(nan_matrix.T, cmap=mpl.colormaps["hsv"])
            plt.colorbar(
                location="top",
                label="angle of incidence of self echo",
            )
            plt.ylabel("self echo reflected from")
            plt.xlabel("interpulse interval")
            print(nan_matrix.shape)
            print(len(nan_matrix_rows))
            # plt.yticks(labels=nan_matrix_rows, ticks=np.arange(0, nan_matrix.shape[1]))
            plt.yticks(
                labels=track_unique_reflections, ticks=np.arange(0, nan_matrix.shape[1])
            )
            plt.title(f"{NUM_BATS} bats")
            plt.clim(-np.pi, np.pi)
            # plt.show()
            dir_to_store = DIRECTORY_STORE_PLOTS + f"/{NUM_BATS}/{FOCAL_BAT}/"
            make_dir(dir_to_store)
            plt.show()
            plt.clf()



NameError: name 'INCLUDE_DIRECT_SOUNDS_ARR' is not defined